In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from tensorflow.keras.layers import Input,Dense,Conv2D, Flatten,Dropout,BatchNormalization, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
import cv2
import os

In [3]:
train_path = '/Users/akashzamnani/datasets/image-classification/simpson/simpsons_dataset'
test_path = '/Users/akashzamnani/datasets/image-classification/simpson/kaggle_simpson_testset'

In [4]:
train_img = [os.path.join(train_path,f) for f in os.listdir(train_path)]
test_img = [os.path.join(test_path,f) for f in os.listdir(test_path)]

In [5]:
df_train = pd.read_csv('/Users/akashzamnani/datasets/image-classification/simpson/number_pic_char.csv')

In [7]:
df_train.head

<bound method NDFrame.head of     Unnamed: 0                      name  total  train  test  bounding_box
0            0             Homer Simpson   2246   1909   337           612
1            1              Ned Flanders   1454   1236   218           595
2            2               Moe Szyslak   1452   1234   218           215
3            3              Lisa Simpson   1354   1151   203           562
4            4              Bart Simpson   1342   1141   201           554
5            5             Marge Simpson   1291   1097   194           557
6            6          Krusty The Clown   1206   1025   181           226
7            7         Principal Skinner   1194   1015   179           506
8            8  Charles Montgomery Burns   1193   1014   179           650
9            9       Milhouse Van Houten   1079    917   162           210
10          10              Chief Wiggum    986    838   148           209
11          11    Abraham Grampa Simpson    913    776   137          

In [8]:
df_train['name'].unique()

array(['Homer Simpson', 'Ned Flanders', 'Moe Szyslak', 'Lisa Simpson',
       'Bart Simpson', 'Marge Simpson', 'Krusty The Clown',
       'Principal Skinner', 'Charles Montgomery Burns',
       'Milhouse Van Houten', 'Chief Wiggum', 'Abraham Grampa Simpson',
       'Sideshow Bob', 'Apu Nahasapeemapetilon', 'Kent Brockman',
       'Comic Book Guy', 'Edna Krabappel', 'Nelson Muntz',
       'Lenny Leonard', 'Mayor Quimby', 'Waylon Smithers',
       'Maggie Simpson', 'Groundskeeper Willie', 'Barney Gumble',
       'Selma Bouvier', 'Carl Carlson', 'Ralph Wiggum', 'Patty Bouvier',
       'Martin Prince', 'Professor John Frink', 'Snake Jailbird',
       'Cletus Spuckler', 'Rainier Wolfcastle', 'Agnes Skinner',
       'Sideshow Mel', 'Otto Mann', 'Fat Tony', 'Gil', 'Miss Hoover',
       'Disco Stu', 'Troy Mcclure', 'Lionel Hutz', 'Jimbo Jones',
       'Bumblebee Man', 'Hans Moleman', 'Helen Lovejoy', 'Jasper Beardly'],
      dtype=object)

In [16]:
# df_train = df_train.dropna()
df_train.name.unique

<bound method Series.unique of 0                Homer Simpson
1                 Ned Flanders
2                  Moe Szyslak
3                 Lisa Simpson
4                 Bart Simpson
5                Marge Simpson
6             Krusty The Clown
7            Principal Skinner
8     Charles Montgomery Burns
9          Milhouse Van Houten
10                Chief Wiggum
11      Abraham Grampa Simpson
12                Sideshow Bob
13      Apu Nahasapeemapetilon
14               Kent Brockman
15              Comic Book Guy
16              Edna Krabappel
17                Nelson Muntz
18               Lenny Leonard
19                Mayor Quimby
20             Waylon Smithers
21              Maggie Simpson
22        Groundskeeper Willie
23               Barney Gumble
24               Selma Bouvier
25                Carl Carlson
26                Ralph Wiggum
27               Patty Bouvier
28               Martin Prince
29        Professor John Frink
30              Snake Jailbird
31      

In [15]:
K = len(df_train['name'].unique())
K

47

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 25,
    width_shift_range=.1,
    height_shift_range=.1,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split = 0.2
)

In [18]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(80,80),
    batch_size=150,
    subset='training',
    class_mode='categorical',
    color_mode = 'rgb'
)

Found 33511 images belonging to 43 classes.


In [19]:
val_datagen = ImageDataGenerator(rescale=1./255,validation_split=.2)

In [20]:
val_generator = val_datagen.flow_from_directory(
    train_path,
    target_size=(80,80),
    batch_size=150,
    subset = 'validation',
    class_mode='categorical',
    color_mode='rgb'
)

Found 8355 images belonging to 43 classes.


In [21]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [23]:
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(80,80),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)

Found 990 images belonging to 1 classes.


In [37]:


i = Input(shape=(80,80,3))
x = Conv2D(32,(3,3),activation='relu',padding='same')(i)
x = Conv2D(32,(3,3),activation='relu')(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = Dropout(.2)(x)
x = Conv2D(64,(3,3),activation='relu',padding='same')(x)
x = Conv2D(64,(3,3),activation='relu')(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = Dropout(.2)(x)           
x = Conv2D(256,(3,3),activation='relu',padding='same')(x)
x = Conv2D(256,(3,3),activation='relu')(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = Dropout(.2)(x)
x = Flatten()(x)          
x = Dense(1024,activation='relu')(x)
x = Dropout(.5)(x)
x = Dense(43,activation='softmax')(x)
              
model = Model(i,x)



In [38]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 80, 80, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 80, 80, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 78, 78, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 39, 39, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 39, 39, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 39, 39, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 37, 37, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 18, 18, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 18, 18, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 18, 18, 256)    │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │    16,778,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 43)             │        44,075 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,625,675 (67.24 MB)

 Trainable params: 17,625,675 (67.24 MB)

 Non-trainable params: 0 (0.00 B)

In [39]:
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])

In [40]:
callbacks = [
tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
    min_delta = 0.001,
    mode="max",
    patience = 10,
    verbose = 1,
    restore_best_weights = True),
    
tf.keras.callbacks.ModelCheckpoint(
    filepath='./model.keras',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)]

In [41]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    batch_size=300,
    callbacks=callbacks,
    epochs = 3
)

Epoch 1/3


/Users/akashzamnani/anaconda3/envs/py3-TF2.0/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


224/224 ━━━━━━━━━━━━━━━━━━━━ 239s 1s/step - accuracy: 0.4453 - loss: 2.6869 - val_accuracy: 0.5010 - val_loss: 2.2543
Epoch 2/3
224/224 ━━━━━━━━━━━━━━━━━━━━ 239s 1s/step - accuracy: 0.5026 - loss: 2.2645 - val_accuracy: 0.5010 - val_loss: 2.2508
Epoch 3/3
224/224 ━━━━━━━━━━━━━━━━━━━━ 239s 1s/step - accuracy: 0.5018 - loss: 2.1449 - val_accuracy: 0.5070 - val_loss: 1.9931
Restoring model weights from the end of the best epoch: 1.
